In [1]:
import re
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.models import Sequential
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical

import numpy as np

import pandas as pd


In [2]:
# carregar modelo word2vec
word2vec = KeyedVectors.load_word2vec_format('skip_s100.txt')
vocab_size, embedding_size = word2vec.vectors.shape

In [3]:
# carregar dados de treinamento do CSV
df = pd.read_csv('train.csv', header=None, names=['texto', 'classe'])
X = df['texto'].values
y = df['classe'].values

In [4]:
# tokenizar o texto
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(X)
X = tokenizer.texts_to_sequences(X)

# padronizar o tamanho das sequencias
max_length = max([len(text) for text in X])
X = pad_sequences(X, maxlen=max_length, padding='post')

In [5]:
# construir modelo
model = Sequential()
model.add(Embedding(vocab_size, embedding_size, weights=[word2vec.vectors], input_length=max_length, trainable=False))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64))
model.add(Dropout(0.2))
model.add(Dense(7, activation='softmax'))

In [6]:
# compilar modelo
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [7]:
# separar dados em treinamento e validação
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
y_train = to_categorical(y_train, 7)
y_val = to_categorical(y_val, 7)

In [8]:
# treinar modelo
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

Epoch 1/10
26/26 [==============================] - 4s 42ms/step - loss: 1.6016 - accuracy: 0.5616 - val_loss: 0.9823 - val_accuracy: 0.6059
Epoch 2/10
26/26 [==============================] - 0s 16ms/step - loss: 0.6018 - accuracy: 0.7722 - val_loss: 0.4779 - val_accuracy: 0.8424
Epoch 3/10
26/26 [==============================] - 0s 14ms/step - loss: 0.2974 - accuracy: 0.9126 - val_loss: 0.2310 - val_accuracy: 0.9360
Epoch 4/10
26/26 [==============================] - 0s 13ms/step - loss: 0.2103 - accuracy: 0.9261 - val_loss: 0.2895 - val_accuracy: 0.8916
Epoch 5/10
26/26 [==============================] - 0s 12ms/step - loss: 0.1038 - accuracy: 0.9717 - val_loss: 0.1566 - val_accuracy: 0.9458
Epoch 6/10
26/26 [==============================] - 0s 12ms/step - loss: 0.0820 - accuracy: 0.9778 - val_loss: 0.1260 - val_accuracy: 0.9754
Epoch 7/10
26/26 [==============================] - 0s 12ms/step - loss: 0.1157 - accuracy: 0.9680 - val_loss: 0.1808 - val_accuracy: 0.9409
Epoch 8/10
26